# Import library

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time

# Load three csv files
Summary: 937 unique drivers and 193502 unique rides in total

In [2]:
driver_df = pd.read_csv('driver_ids.csv')
ride_df = pd.read_csv('ride_ids.csv')
ride_timestamps_df = pd.read_csv('ride_timestamps.csv')

In [3]:
'''
Get the shape of each dataframe
'''
print('driver ids:',driver_df.shape)
display(driver_df.head())
print('ride ids:',ride_df.shape)
display(ride_df.head())
print('ride timestamps:',ride_timestamps_df.shape)
display(ride_timestamps_df.head())

driver ids: (937, 2)


,driver_id,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00
2,011e5c5dfc5c2c92501b8b24d47509bc,2016-04-05 00:00:00
3,0152a2f305e71d26cc964f8d4411add9,2016-04-23 00:00:00
4,01674381af7edd264113d4e6ed55ecda,2016-04-29 00:00:00


ride ids: (193502, 5)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100


ride timestamps: (970405, 3)


,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


# Inspect Nan and abnormal values

In [4]:
'''
Nan value inspection
'''
print('driver ids info:------------------------------')
driver_df.info()
print('ride ids info:--------------------------------')
ride_df.info()
print('ride timestamps info:-------------------------')
ride_timestamps_df.info()
'''
ride_timestamps has one Nan value in the column timestamp
    TODO: delete this ride or fill it with an artificial value?
'''
display(ride_timestamps_df[ride_timestamps_df.isnull().any(axis=1)])

driver ids info:------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 2 columns):
driver_id              937 non-null object
driver_onboard_date    937 non-null object
dtypes: object(2)
memory usage: 14.7+ KB
ride ids info:--------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193502 entries, 0 to 193501
Data columns (total 5 columns):
driver_id          193502 non-null object
ride_id            193502 non-null object
ride_distance      193502 non-null int64
ride_duration      193502 non-null int64
ride_prime_time    193502 non-null int64
dtypes: int64(3), object(2)
memory usage: 7.4+ MB
ride timestamps info:-------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970405 entries, 0 to 970404
Data columns (total 3 columns):
ride_id      970405 non-null object
event        970405 non-null object
timestamp    970404 non-null object
dtypes: object(3)
memory usage: 22.2+ MB


,ride_id,event,timestamp
434222,72f0fa0bd86800e9da5c4dced32c8735,arrived_at,NaN


In [5]:
'''
Abnormal value inspection
'''
display(driver_df.describe())
display(ride_df.describe())
display(ride_timestamps_df.describe())

,driver_id,driver_onboard_date
count,937,937
unique,937,49
top,9494350df132e6748afca3bc5d138dcc,2016-04-05 00:00:00
freq,1,36


,ride_distance,ride_duration,ride_prime_time
count,193502.000000,193502.000000,193502.000000
mean,6955.218266,858.966099,17.305893
std,8929.444606,571.375818,30.825800
min,-2.000000,2.000000,0.000000
25%,2459.000000,491.000000,0.000000
50%,4015.000000,727.000000,0.000000
75%,7193.000000,1069.000000,25.000000
max,724679.000000,28204.000000,500.000000


,ride_id,event,timestamp
count,970405,970405,970404
unique,194081,5,865826
top,43b8fb03256e30a1a93db73d09a05b26,requested_at,2016-04-30 22:09:15
freq,5,194081,7


In [6]:
'''
TODO: Need to think about how to deal with this case, why will ride_distance <= 0?
TODO: the number of ride_id in ride_df and that of ride_timestamps doesn't fit (193502 vs 194081)
'''
abnormal_ride_df = ride_df[ride_df.ride_distance <= 0]
print(abnormal_ride_df.shape)
display(abnormal_ride_df.head())

(187, 5)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
3557,04d686ef2fba281e395992a28f14fdd3,c819e844b0c043786f1291e466478ca0,0,127,0
4221,0656192a402808805282e60761bda088,2f4fe99f74f1d71fdf2e8e18da9670c0,0,404,25
5120,07dd442e3e0b9f0f9b0d69c7b47cbb06,ad2b44316299477f2d6537922af22bf4,0,7,0
7144,0b631e16fa61f7321da18cf35a076d5f,1410283d27fdcf6c25a11f856ce5cde0,0,66,0
7713,0c02bd2b09f7193103279ab9b760b777,9fd795fdb616bd47c41e1ad07abbd6f3,0,23,50


In [7]:
'''
find overlap of driver_id between dirver_df and ride_df
TODO: some drivers don't have ride information--->delete? (937 vs 854)
'''
print(len(set(driver_df.driver_id.unique()).intersection(set(ride_df.driver_id.unique()))))

854


In [8]:
'''
find overlap of ride_id between ride_df and ride_timestamps_df
TODO: some rides don't have ride timestamps--->delete? (193502 vs 184819)
'''
print(len(set(ride_df.ride_id.unique()).intersection(set(ride_timestamps_df.ride_id.unique()))))

184819


# Merge all dfs to one df

In [9]:
'''
merge driver_df and ride_df (Get intersection based on driver_id)
'''
big_df = ride_df.merge(driver_df,left_on='driver_id',right_on='driver_id')
print(big_df.shape)
display(big_df.head())

(185891, 6)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29 00:00:00
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 00:00:00
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29 00:00:00
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29 00:00:00
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29 00:00:00


In [10]:
# get overlapped ride_id between big_df and ride_timestamps_df
big_df = big_df[big_df['ride_id'].isin(ride_timestamps_df.ride_id.unique())]
big_df.reset_index(drop=True,inplace=True)
print(big_df.shape)
display(big_df.head())

(184209, 6)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29 00:00:00
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 00:00:00
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29 00:00:00
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29 00:00:00
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29 00:00:00


In [11]:
start = time.time()
# for each unique ride id in big_df
for idx in range(big_df.shape[0]):
    rideid = big_df.iloc[idx]['ride_id']
    # first find rideid timestamps info in ride_timestamps_df
    target = ride_timestamps_df[ride_timestamps_df.ride_id == rideid]
    # for each (event,timestamp) pair
    for (e,t) in zip(list(target.event),list(target.timestamp)):
        big_df.at[idx,e] = t
    # double check index
    if big_df[big_df.ride_id == rideid]['requested_at'].values[0] != \
    ride_timestamps_df[ride_timestamps_df.ride_id == rideid].iloc[0,-1]:
        print(idx)
print('duration:',(time.time()-start)/3600,'hrs')

duration: 7.626359305050638 hrs


In [12]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184209 entries, 0 to 184208
Data columns (total 11 columns):
driver_id              184209 non-null object
ride_id                184209 non-null object
ride_distance          184209 non-null int64
ride_duration          184209 non-null int64
ride_prime_time        184209 non-null int64
driver_onboard_date    184209 non-null object
requested_at           184209 non-null object
accepted_at            184209 non-null object
arrived_at             184208 non-null object
picked_up_at           184209 non-null object
dropped_off_at         184209 non-null object
dtypes: int64(3), object(8)
memory usage: 15.5+ MB


In [13]:
# saved for future use
big_df.to_csv('merged_big_driver_ride_df.csv',index=False)

# Start to work on calculating extra variables

In [14]:
test1 = pd.read_csv('merged_big_driver_ride_df.csv')
print(test1.shape)
display(test1.head())

(184209, 11)


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-03-29 00:00:00,2016-04-23 02:13:50,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:16:40,2016-04-23 02:22:07
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 00:00:00,2016-03-29 19:00:49,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:04:01,2016-03-29 19:17:30
2,002be0ffdc997bd5c50703158b7c2491,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-03-29 00:00:00,2016-06-21 11:56:31,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:01:35,2016-06-21 12:11:07
3,002be0ffdc997bd5c50703158b7c2491,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-03-29 00:00:00,2016-05-19 09:15:29,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 09:18:20,2016-05-19 10:13:58
4,002be0ffdc997bd5c50703158b7c2491,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-03-29 00:00:00,2016-04-20 22:05:30,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:07:02,2016-04-20 22:20:45


In [15]:
'''
validate the correctness of combined df by randomly selecting ride ids to verify
'''
ids = test1.ride_id
i = np.random.choice(ids,10)
for x in i:
    display(test1[test1.ride_id == x])
    display(ride_timestamps_df[ride_timestamps_df.ride_id == x])

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
123465,a4401d3bb25a651812f6781fcce5df77,265613fa62b9a1f16fab87573e7db6a6,3232,531,50,2016-04-14 00:00:00,2016-06-07 22:30:36,2016-06-07 22:30:40,2016-06-07 22:35:14,2016-06-07 22:35:16,2016-06-07 22:44:07


,ride_id,event,timestamp
144610,265613fa62b9a1f16fab87573e7db6a6,requested_at,2016-06-07 22:30:36
144611,265613fa62b9a1f16fab87573e7db6a6,accepted_at,2016-06-07 22:30:40
144612,265613fa62b9a1f16fab87573e7db6a6,arrived_at,2016-06-07 22:35:14
144613,265613fa62b9a1f16fab87573e7db6a6,picked_up_at,2016-06-07 22:35:16
144614,265613fa62b9a1f16fab87573e7db6a6,dropped_off_at,2016-06-07 22:44:07


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
92297,75ff47d4ba4bd4480629671a666500ba,e920e35edbfc4c5cef2edec6d1022981,9143,1584,0,2016-03-28 00:00:00,2016-06-26 16:35:58,2016-06-26 16:36:06,2016-06-26 16:41:22,2016-06-26 16:41:24,2016-06-26 17:07:48


,ride_id,event,timestamp
883355,e920e35edbfc4c5cef2edec6d1022981,requested_at,2016-06-26 16:35:58
883356,e920e35edbfc4c5cef2edec6d1022981,accepted_at,2016-06-26 16:36:06
883357,e920e35edbfc4c5cef2edec6d1022981,arrived_at,2016-06-26 16:41:22
883358,e920e35edbfc4c5cef2edec6d1022981,picked_up_at,2016-06-26 16:41:24
883359,e920e35edbfc4c5cef2edec6d1022981,dropped_off_at,2016-06-26 17:07:48


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
90496,74de6f7e57690a1687935227360edf7f,63d9b91dbc77f207f53b60aba5bfa08a,4904,673,0,2016-05-05 00:00:00,2016-06-18 13:28:34,2016-06-18 13:28:38,2016-06-18 13:30:44,2016-06-18 13:31:43,2016-06-18 13:42:56


...

,ride_id,event,timestamp
758335,c855003adff572d2863ad000bdb38c4e,requested_at,2016-04-20 17:14:42
758336,c855003adff572d2863ad000bdb38c4e,accepted_at,2016-04-20 17:14:46
758337,c855003adff572d2863ad000bdb38c4e,arrived_at,2016-04-20 17:24:06
758338,c855003adff572d2863ad000bdb38c4e,picked_up_at,2016-04-20 17:24:10
758339,c855003adff572d2863ad000bdb38c4e,dropped_off_at,2016-04-20 17:42:40


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
3924,05addf442c147875efa5cf53453ad47b,968ab5600d78ae369765ccb70796c49e,2284,1272,0,2016-04-08 00:00:00,2016-04-30 14:05:44,2016-04-30 14:05:48,2016-04-30 14:10:12,2016-04-30 14:10:15,2016-04-30 14:31:27


,ride_id,event,timestamp
570505,968ab5600d78ae369765ccb70796c49e,requested_at,2016-04-30 14:05:44
570506,968ab5600d78ae369765ccb70796c49e,accepted_at,2016-04-30 14:05:48
570507,968ab5600d78ae369765ccb70796c49e,arrived_at,2016-04-30 14:10:12
570508,968ab5600d78ae369765ccb70796c49e,picked_up_at,2016-04-30 14:10:15
570509,968ab5600d78ae369765ccb70796c49e,dropped_off_at,2016-04-30 14:31:27


,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time,driver_onboard_date,requested_at,accepted_at,arrived_at,picked_up_at,dropped_off_at
66916,5a880bc9cddbb3845438de87f859c360,ba7ecae41e107acb84932e7fbe8ced86,1684,466,0,2016-04-19 00:00:00,2016-05-03 18:25:57,2016-05-03 18:26:00,2016-05-03 18:31:03,2016-05-03 18:31:06,2016-05-03 18:38:52


,ride_id,event,timestamp
706420,ba7ecae41e107acb84932e7fbe8ced86,requested_at,2016-05-03 18:25:57
706421,ba7ecae41e107acb84932e7fbe8ced86,accepted_at,2016-05-03 18:26:00
706422,ba7ecae41e107acb84932e7fbe8ced86,arrived_at,2016-05-03 18:31:03
706423,ba7ecae41e107acb84932e7fbe8ced86,picked_up_at,2016-05-03 18:31:06
706424,ba7ecae41e107acb84932e7fbe8ced86,dropped_off_at,2016-05-03 18:38:52
